# Test Data Analysis from Optimizations

By Carla Nannini

## Introduction


In [29]:
# Imports
import numpy as np
import matplotlib.pyplot as plt   # allows to plot charts
import pandas as pd              
import seaborn as sns             # python data visualization library based on matplotlib
import glob
import pickle
from os import listdir, rename
import os
import re
# To sort tests per test_id
from collections import OrderedDict


# To be allow to zoom in on the plots, uncomment the following lines
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

sns.set(style="darkgrid")

In [30]:
# Functions

# Get dictionary keys as list
def getList(dict):
    return dict.keys()

# Return a color map to assign colors to test_ids
def get_cmap(n, name='gist_rainbow'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

The next code is used so as to retrieve the `.pkl` files from the optimizations. 
For example, with the first Experiment (cmaes_spinal_3), the tunable hyperparameters chosen are the following:
`-g` 150, `-sig` 0.6, `-init_speed` 1.7, `-tgt_speed` 1.4, `-file` control/params_2D.txt. 

The 150 generations run and lead to 150 `.pkl` files. 
We want to retrieve the information in all these files so as to select the best generation and perform data analysis on this.

In [109]:
# To rename all files of format '_x.pkl' to format 'x.pkl'
#for file in os.listdir(path_dir):
    #if "_" in file:
    #    original = os.path.join(path_dir, file)
     #   renamed = os.path.join(path_dir, file.replace("_", ""))
     #   os.rename(original, renamed)
        
path = "/home/efx/Desktop/cespar_exo_opensimrl/Optimization_Results/cmaes_spinal_3/Results"

# Loop to recursively look for files with .pkl extension, even in subfolders
all_files = {}
for root, dirs, files in os.walk(path):
    for file in files:
        if "_" in file:
            original = os.path.join(path, file)
            rename = os.path.join(path, file.replace("_", ""))
            os.rename(original, rename)

for root, dirs, files in os.walk(path):
    for file in files:
        test_id = os.path.splitext(file)[0]
        #print(test_id)
        all_files[test_id] = []
        #print(file)
        new_path = "{}".format(file)
        all_files[test_id].append(new_path)

print(type(all_files))
print(type(all_files.items()))
print(all_files.keys())
print(type(all_files.keys()))
all_files = OrderedDict(sorted(all_files, key=lambda x: int(x)))
print(all_files)




# Open and load the files as dictionaries
# We store those dict in an bigger dict for future reference, the filename will be the key



#for test_id, files in all_files.items():
    #if current_test != test_id:
    #    current_test = test_id
     #   data[current_test] = {}
    #    test_color[current_test] = possibleColors(count)
     #   count += 1
    #for pkl in files:
     #   with open(pkl, 'rb') as f:
     #       data[current_test][pkl.split(os.sep)[-1].split('.')[-2].split('_')[-1]] = pickle.load(f)

            
            
#print(type[data])
##### Data structure #####
# 1 -> data['1']['logbook']

##### Data structure #####
# values -> data['test_number']['values']
# reflex -> data['test_number']['reflex']
# muscle -> data['test_number']['muscle]
# noise -> data['test_number']['noise']
    



#for curr_file in all_files:
#    with open(curr_file, 'rb') as f:
#        mydata = pickle.load(f)#

#print(all_files['logbook'])

dict_items([('139', ['139.pkl']), ('7', ['7.pkl']), ('110', ['110.pkl']), ('46', ['46.pkl']), ('12', ['12.pkl']), ('40', ['40.pkl']), ('141', ['141.pkl']), ('63', ['63.pkl']), ('72', ['72.pkl']), ('53', ['53.pkl']), ('145', ['145.pkl']), ('126', ['126.pkl']), ('140', ['140.pkl']), ('68', ['68.pkl']), ('76', ['76.pkl']), ('2', ['2.pkl']), ('3', ['3.pkl']), ('59', ['59.pkl']), ('79', ['79.pkl']), ('84', ['84.pkl']), ('97', ['97.pkl']), ('5', ['5.pkl']), ('122', ['122.pkl']), ('17', ['17.pkl']), ('55', ['55.pkl']), ('32', ['32.pkl']), ('114', ['114.pkl']), ('95', ['95.pkl']), ('129', ['129.pkl']), ('137', ['137.pkl']), ('92', ['92.pkl']), ('42', ['42.pkl']), ('135', ['135.pkl']), ('10', ['10.pkl']), ('70', ['70.pkl']), ('19', ['19.pkl']), ('118', ['118.pkl']), ('138', ['138.pkl']), ('4', ['4.pkl']), ('54', ['54.pkl']), ('104', ['104.pkl']), ('71', ['71.pkl']), ('130', ['130.pkl']), ('98', ['98.pkl']), ('48', ['48.pkl']), ('34', ['34.pkl']), ('60', ['60.pkl']), ('36', ['36.pkl']), ('11', [

ValueError: need more than 1 value to unpack

Once we retrieve all the information that is needed from the optimization, we have selected the best generation and ran a local simulation with this best generation. 

From this, we can perform data analysis and retrieve the 18 muscle activities during the entire duration of the simulation as well as the joint angles and velocities.

Next, we will retrieve the muscle stimulus coming to the muscles from the CMAES algorithm.

In [26]:
path_dir = "/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2"

# Loop to recursively look for files with .pkl extension, even in subfolders
pkl_files = {}
for root, dirs, files in os.walk(path_dir):
    folder_name_splitted = root.split(os.sep)[-1].split('_')
    if(len(folder_name_splitted) == 2 and folder_name_splitted[0] == "Best"):
        test_id = folder_name_splitted[1]
        pkl_files[test_id] = []
        for file in files:
            if file.split('_')[-1:][0] in ["values.pkl", "reflex.pkl", "muscle.pkl", "noise.pkl"]:
                path = "{}{}{}".format(root, os.sep, file)
                pkl_files[test_id].append(path)

# Sort files per test_id asc
pkl_files = OrderedDict(sorted(pkl_files.items()))
print(type(pkl_files))
print(pkl_files)

# Color Map, assign a color to a test_id. The first test_id will be represented with the first color of the array aso...
possibleColors = get_cmap(len(pkl_files.keys()) + 1)
test_color = {}

# Open and load the files as dictionaries
# We store those dict in an bigger dict for future reference, the filename will be the key
current_test = None
data = {}
count = 0
for test_id, files in pkl_files.items():
    if current_test != test_id:
        current_test = test_id
        data[current_test] = {}
        test_color[current_test] = possibleColors(count)
        count += 1
    for pkl in files:
        with open(pkl, 'rb') as f:
            data[current_test][pkl.split(os.sep)[-1].split('.')[-2].split('_')[-1]] = pickle.load(f)

##### Data structure #####
# values -> data['test_number']['values']
# reflex -> data['test_number']['reflex']
# muscle -> data['test_number']['muscle]
# noise -> data['test_number']['noise']


<class 'collections.OrderedDict'>
OrderedDict([('1', ['/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_1/cmaes_spinal_4_1_noise.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_1/cmaes_spinal_4_1_reflex.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_1/cmaes_spinal_4_1_muscle.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_1/cmaes_spinal_4_1_values.pkl']), ('148', ['/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_148/cmaes_spinal_4_148_muscle.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_148/cmaes_spinal_4_148_noise.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_148/cmaes_spinal_4_148_values.pkl', '/home/efx/Desktop/cespar_exo_opensimrl/logs/simulation_data/RFX_Exp2/Best_148/cmaes_spinal_4_148_reflex.pkl'])])


In [ ]:
# Interpolation for all cycles
def getNormalizedData(data, start_cycles, end_cycles):
    Y = []
    N = 100
    for start_cycle, end_cycle in zip(start_cycles, end_cycles):
        yp = data[start_cycle:end_cycle]
        xp = range(len(yp))
        x = np.linspace(0,len(yp),N)
        y = np.interp(x, xp, yp)
        Y.append(y)
    Y = np.array(Y).transpose()
    return Y

# Interpolation for one cycle
def NormalizationOneCycle(data, start_cycle_left):
    N = 100
    yp = data[start_cycle_left[0]:start_cycle_left[1]-1]
    xp = range(len(yp))
    x = np.linspace(0,len(yp),N)
    y = np.interp(x, xp, yp)

# Plot with the standard deviation around the mean
def plotWithStd(normalized_data, title, xlabel, ylabel, legendlabel, color, fontsize=15):
    m = np.mean(normalized_data,1)
    s = np.std(normalized_data,1)
    LENGTH = normalized_data.shape[0]
    
    plt.plot(m, label=legendlabel, color=color)
    plt.suptitle(title, fontsize=fontsize)
    plt.xlabel(xlabel, fontsize=fontsize)
    plt.ylabel(ylabel, fontsize=fontsize)
    plt.legend()
    plt.fill_between(np.linspace(0,LENGTH,LENGTH),m-s, m+s ,alpha=0.3, color=color)
    
# Plot the standard deviation around the mean, for the vertical line showing the end of the stance phase
def plotStdStance(normalized_data, ymin, ymax, color, linestyle, fontsize):
    m = np.mean(normalized_data)
    max = np.max(normalized_data)
    min = np.min(normalized_data)
    plt.axvline(min, alpha=0, color=color)
    plt.axvline(max, alpha=0, color=color)
    plt.axvline(m, color=color)
    plt.fill_between([min, max], [ymax, ymax], alpha=0.3, color=color)

# Plots with the standard deviation around the mean for real human data
def plotMeanStd(norm_mean, norm_min, norm_max, title, xlabel, ylabel, legendlabel, legendLoc, color):
    LENGTH = norm_mean.shape[0]
    plt.plot(norm_mean, label=legendlabel, color=color)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc = legendLoc)
    plt.fill_between(np.linspace(0, LENGTH, LENGTH), norm_min, norm_max ,alpha=0.3, color=color)
    
# Cast value val to int
def CastValInt(dataframe):
    dataframe['index'] = dataframe['index'].astype(int)

# Cast value val to float
def CastValFloat(dataframe):
    dataframe['norm_min'] = dataframe['norm_min'].astype(float)
    dataframe['norm_mean'] = dataframe['norm_mean'].astype(float)
    dataframe['norm_max'] = dataframe['norm_max'].astype(float)

In [ ]:
real_data_dir = '/home/efx/Desktop/cespar_exo_opensimrl/RealHumanData'

HipFlexion = pd.read_excel('{}{}HipFlexion.ods'.format(real_data_dir, os.sep), engine='odf')
KneeFlexion = pd.read_excel('{}{}KneeFlexion.ods'.format(real_data_dir, os.sep), engine='odf')
AnkleFlexion = pd.read_excel('{}{}AnkleFlexion.ods'.format(real_data_dir, os.sep), engine='odf')
GroundReactionForces = pd.read_excel('{}{}GroundReactionForces.ods'.format(real_data_dir, os.sep), engine='odf')

# Data formatting
HipFlex = pd.DataFrame(HipFlexion, columns=['index', 'norm_min', 'norm_mean', 'norm_max'])
KneeFlex = pd.DataFrame(KneeFlexion, columns=['index', 'norm_min', 'norm_mean', 'norm_max'])
AnkleFlex = pd.DataFrame(AnkleFlexion, columns=['index', 'norm_min', 'norm_mean', 'norm_max'])
GRF = pd.DataFrame(GroundReactionForces, columns=['index', 'norm_min', 'norm_mean', 'norm_max'])

# Casting
CastValInt(HipFlex)
CastValFloat(HipFlex)
CastValInt(KneeFlex)
CastValFloat(KneeFlex)
CastValInt(AnkleFlex)
CastValFloat(AnkleFlex)
CastValInt(GRF)
CastValFloat(GRF)

GRF.head()

## HIP JOINT ANGLE formatting 
HipFlex_mean = HipFlex['norm_mean']
HipFlex_min = HipFlex['norm_min']
HipFlex_max = HipFlex['norm_max']

# KNEE JOINT ANGLE formatting
KneeFlex_mean = KneeFlex['norm_mean']
KneeFlex_min = KneeFlex['norm_min']
KneeFlex_max = KneeFlex['norm_max']

# ANKLE JOINT ANGLE formatting
AnkleFlex_mean = AnkleFlex['norm_mean']
AnkleFlex_min = AnkleFlex['norm_min']
AnkleFlex_max = AnkleFlex['norm_max']

# GROUND REACTION FORCES formatting
GRF_mean = GRF['norm_mean']
GRF_min = GRF['norm_min']
GRF_max = GRF['norm_max']

In [ ]:
# Retrieval of the ground reaction forces along y-axis
grf_axis_y_rleg = {}
grf_axis_y_lleg = {}

for test_id, test in data.items():
    grf_axis_y_rleg[test_id] = [test['values'][i]["r_leg"]["ground_reaction_forces"][2] for i in range(len(test['values']))]
    grf_axis_y_lleg[test_id] = [test['values'][i]["l_leg"]["ground_reaction_forces"][2] for i in range(len(test['values']))]

# Plots of the GRF for the right and left leg separated
for test_id, grf in grf_axis_y_rleg.items(): 
    plt.axhline(y=0.3, color='xkcd:coral', linestyle='-')
    plt.plot(np.array(grf)[450:450+125,], color=test_color[test_id], label='Exp {}'.format(test_id))
    plt.title('Ground Reaction Force for right leg - Test {}'.format(test_id))
    plt.ylabel('GRF [%BW]')
    plt.xlabel('One gait cycle')
    plt.legend(bbox_to_anchor = (1.05, 1), loc='upper left')
# To display all plot independently, please put the plt.show method back in the for loop as below
    plt.show()

for test_id, grf in grf_axis_y_lleg.items(): 
    plt.plot(np.array(grf)[450:450+125,], color=test_color[test_id], label='Exp {}'.format(test_id))
    plt.axhline(y=0.3, color='xkcd:coral', linestyle='-')
    plt.title('Ground Reaction Force for left leg - Test {}'.format(test_id))
    plt.ylabel('GRF [%BW]')
    plt.xlabel('One gait cycle')
    plt.legend(bbox_to_anchor = (1.05, 1), loc='upper left')
    plt.show()

In [ ]:
# Separation in cycle + normalization of data + mean and std plot
GRF_Y_left = {}
GRF_Y_right = {}
for test_id, grf in grf_axis_y_lleg.items():
    GRF_Y_left[test_id] = np.array(grf)
for test_id, grf in grf_axis_y_rleg.items():
    GRF_Y_right[test_id] = np.array(grf)

START_CYCLE_l = {}
START_CYCLE_r = {}
for test_id, grf in GRF_Y_left.items():
    START_CYCLE_l[test_id] = np.where(np.diff(1.0*(grf > 0.3)) == 1)[0]
for test_id, grf in GRF_Y_right.items():
    START_CYCLE_r[test_id] = np.where(np.diff(1.0*(grf > 0.3)) == 1)[0]

dt = 0.01

# CYCLE DURATIONS FOR EACH SIMULATION
cycle_durations_left = {}
cycle_durations_right = {}
for test_id, start_cycles in START_CYCLE_l.items():
    cycle_durations_left[test_id] = (start_cycles[1:] - start_cycles[:-1])*dt   # in seconds
for test_id, start_cycles in START_CYCLE_r.items():
    cycle_durations_right[test_id] = (start_cycles[1:] - start_cycles[:-1])*dt  # in seconds
    
# Mean duration for each leg
mean_duration_left = {}
mean_duration_right = {}
for test_id, start_cycles in cycle_durations_left.items():
    mean_duration_left[test_id] = np.mean(cycle_durations_left[test_id])
for test_id, start_cycles in cycle_durations_right.items():
    mean_duration_right[test_id] = np.mean(cycle_durations_right[test_id])  
    
# Mean duration overall
mean_duration = {}
for test_id, start_cycles in mean_duration_left.items():
    for test_id, start_cycles in mean_duration_right.items():
        mean_duration[test_id] = (mean_duration_left[test_id] + mean_duration_right[test_id])/2

print(mean_duration)

# STANCES AND SWINGS
LIFT_OFF_l = {}
LIFT_OFF_r = {}
for test_id, grf in GRF_Y_left.items():
    LIFT_OFF_l[test_id] = np.where(np.diff(1.0*(grf > 0.3)) == -1)[0]
for test_id, grf in GRF_Y_right.items():
    LIFT_OFF_r[test_id] = np.where(np.diff(1.0*(grf > 0.3)) == -1)[0]
    
# CONDITIONS
for test_id, start_cycles in START_CYCLE_l.items():
    if(start_cycles[0] > LIFT_OFF_l[test_id][0]):
        LIFT_OFF_l[test_id] = LIFT_OFF_l[test_id][1:]
for test_id, start_cycles in START_CYCLE_r.items():
    if(start_cycles[0] > LIFT_OFF_r[test_id][0]):
        LIFT_OFF_r[test_id] = LIFT_OFF_r[test_id][1:]
for test_id, lift_off in LIFT_OFF_l.items():
    lift_off = lift_off[len(START_CYCLE_l[test_id])-len(lift_off):]
for test_id, lift_off in LIFT_OFF_r.items():
    lift_off = lift_off[len(START_CYCLE_r[test_id])-len(lift_off):]    
for test_id, lift_off in LIFT_OFF_l.items():
    if (len(LIFT_OFF_l[test_id]) > len(cycle_durations_left[test_id])):
        LIFT_OFF_l[test_id] = LIFT_OFF_l[test_id][:-1]
for test_id, lift_off in LIFT_OFF_r.items():
    if (len(LIFT_OFF_r[test_id]) > len(cycle_durations_right[test_id])):
        LIFT_OFF_r[test_id] = LIFT_OFF_r[test_id][:-1]     

# STANCE/SWING durations - LEFT 
stance_durations_left = {}
swing_durations_left = {}
for test_id, lift_off in LIFT_OFF_l.items():
    stance_durations_left[test_id] = (lift_off - START_CYCLE_l[test_id][:len(lift_off)])*dt  # in seconds
    swing_durations_left[test_id] = (START_CYCLE_l[test_id][len(START_CYCLE_l[test_id]) - len(lift_off):] - lift_off)*dt  # in seconds
#print(stance_durations_left)
#print(swing_durations_left)

# STANCE/SWING durations - RIGHT
stance_durations_right = {}
swing_durations_right = {}
for test_id, lift_off in LIFT_OFF_r.items():
    stance_durations_right[test_id] = (lift_off - START_CYCLE_r[test_id][:len(lift_off)])*dt  # in seconds
    swing_durations_right[test_id] = (START_CYCLE_r[test_id][len(START_CYCLE_r[test_id]) - len(lift_off):] - lift_off)*dt  # in seconds
#print(stance_durations_right)
#print(swing_durations_right)

In [ ]:
muscle_activations = {}
for test_id, test in data.items():
    muscle_activations[test_id] = [test['muscle'][i] for i in range(len(data[test_id]['muscle']))]

# Muscle activations from the muscles of the right leg
HFL_r  = {}
GLU_r  = {}
HAM_r  = {}
RF_r   = {}
VAS_r  = {}
BFSH_r = {}
GAS_r  = {}
SOL_r  = {}
TA_r   = {}

for test_id, muscle_activation in muscle_activations.items():
    HFL_r[test_id]  = np.clip([item[2] for item in muscle_activation], 0.01, 1)
    GLU_r[test_id]  = np.clip([item[3] for item in muscle_activation], 0.01, 1)
    HAM_r[test_id]  = np.clip([item[4] for item in muscle_activation], 0.01, 1)
    RF_r[test_id]   = np.clip([item[5] for item in muscle_activation], 0.01, 1)
    VAS_r[test_id]  = np.clip([item[6] for item in muscle_activation], 0.01, 1)
    BFSH_r[test_id] = np.clip([item[7] for item in muscle_activation], 0.01, 1)
    GAS_r[test_id]  = np.clip([item[8] for item in muscle_activation], 0.01, 1)
    SOL_r[test_id]  = np.clip([item[9] for item in muscle_activation], 0.01, 1)
    TA_r[test_id]  =  np.clip([item[10] for item in muscle_activation], 0.01, 1)
    
# Muscle activations from the muscles of the left leg
HFL_l  = {}
GLU_l  = {}
HAM_l  = {}
RF_l   = {}
VAS_l  = {}
BFSH_l = {}
GAS_l  = {}
SOL_l  = {}
TA_l   = {}

for test_id, muscle_activation in muscle_activations.items():
    HFL_l[test_id]  = np.clip([item[13] for item in muscle_activation], 0.01, 1)
    GLU_l[test_id]  = np.clip([item[14] for item in muscle_activation], 0.01, 1)
    HAM_l[test_id]  = np.clip([item[15] for item in muscle_activation], 0.01, 1)
    RF_l[test_id]   = np.clip([item[16] for item in muscle_activation], 0.01, 1)
    VAS_l[test_id]  = np.clip([item[17] for item in muscle_activation], 0.01, 1)
    BFSH_l[test_id] = np.clip([item[18] for item in muscle_activation], 0.01, 1)
    GAS_l[test_id]  = np.clip([item[19] for item in muscle_activation], 0.01, 1)
    SOL_l[test_id]  = np.clip([item[20] for item in muscle_activation], 0.01, 1)
    TA_l[test_id]   = np.clip([item[21] for item in muscle_activation], 0.01, 1)

# Normalized muscle activities involved in the hip only (HFL, GLU)
HFL_r_norm = {}
HFL_l_norm = {}
GLU_r_norm = {}
GLU_l_norm = {}

for test_id, hfl in HFL_r.items():
    HFL_r_norm[test_id] = getNormalizedData(hfl, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, hfl in HFL_l.items():
    HFL_l_norm[test_id] = getNormalizedData(hfl, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)
for test_id, glu in GLU_r.items():
    GLU_r_norm[test_id] = getNormalizedData(glu, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, glu in GLU_l.items():
    GLU_l_norm[test_id] = getNormalizedData(glu, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)   

# Plot of the hip muscle activities for both legs
for test_id in HFL_r_norm.keys():
    #plotWithStd(HFL_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'HFL_r', 'xkcd:red')
    plotWithStd(HFL_l_norm[test_id], 'Hip muscle activities (HFL), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'HFL - Exp {}'.format(test_id), test_color[test_id],15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart    
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()

# Plot of the hip muscle activities for both legs
for test_id in GLU_r_norm.keys():
    #plotWithStd(GLU_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'GLU_r', 'xkcd:orange')
    plotWithStd(GLU_l_norm[test_id], 'Hip muscle activities (GLU), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'GLU - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show() 
    
# Normalized muscle activities involved in the knee only (VAS, BFSH)
VAS_r_norm = {}
VAS_l_norm = {}
BFSH_r_norm = {}
BFSH_l_norm = {}

for test_id, vas in VAS_r.items():
    VAS_r_norm[test_id] = getNormalizedData(vas, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, vas in VAS_l.items():
    VAS_l_norm[test_id] = getNormalizedData(vas, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)
for test_id, bfsh in BFSH_r.items():
    BFSH_r_norm[test_id] = getNormalizedData(bfsh, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, bfsh in BFSH_l.items():
    BFSH_l_norm[test_id] = getNormalizedData(bfsh, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1) 

# Plot of the knee muscle activities for both legs
for test_id in VAS_r_norm.keys():
    #plotWithStd(VAS_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'VAS_r', 'xkcd:red')
    plotWithStd(VAS_l_norm[test_id], 'Knee muscle activities (VAS), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'VAS - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()
    
# Plot of the knee muscle activities for both legs
for test_id in BFSH_r_norm.keys():
    #plotWithStd(BFSH_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'BFSH_r', 'xkcd:orange')
    plotWithStd(BFSH_l_norm[test_id], 'Knee muscle activities (BFSH), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'BFSH - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.show()

# Normalized muscle activities involved in the ankle only (SOL, TA)
SOL_r_norm = {}
SOL_l_norm = {}
TA_r_norm = {}
TA_l_norm = {}

for test_id, sol in SOL_r.items():
    SOL_r_norm[test_id] = getNormalizedData(sol, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, sol in SOL_l.items():
    SOL_l_norm[test_id] = getNormalizedData(sol, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)
for test_id, ta in TA_r.items():
    TA_r_norm[test_id] = getNormalizedData(ta, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, ta in TA_l.items():
    TA_l_norm[test_id] = getNormalizedData(ta, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1) 

# Plot of the ankle muscle activities for both legs
for test_id in SOL_r_norm.keys():
    #plotWithStd(SOL_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'SOL_r', 'xkcd:red')
    plotWithStd(SOL_l_norm[test_id], 'Ankle muscle activities (SOL), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'SOL - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()
    
# Plot of the ankle muscle activities for both legs
for test_id in TA_r_norm.keys():
    #plotWithStd(TA_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'TA_r', 'xkcd:orange')
    plotWithStd(TA_l_norm[test_id], 'Ankle muscle activities (TA), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'TA - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()

# Normalized biarticular muscle activities for the joint hip/knee (HAM, RF)
HAM_r_norm = {}
HAM_l_norm = {}
RF_r_norm = {}
RF_l_norm = {}
for test_id, ham in HAM_r.items():
    HAM_r_norm[test_id] = getNormalizedData(ham, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, ham in HAM_l.items():
    HAM_l_norm[test_id] = getNormalizedData(ham, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)
for test_id, rf in RF_r.items():
    RF_r_norm[test_id] = getNormalizedData(rf, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, rf in RF_l.items():
    RF_l_norm[test_id] = getNormalizedData(rf, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1) 

# Plot of the biarticular muscle activities from joint hip/knee
for test_id in HAM_r_norm.keys():
    #plotWithStd(HAM_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'HAM_r', 'xkcd:red')
    plotWithStd(HAM_l_norm[test_id], 'Biarticular (Hip/Knee) muscle activity (HAM), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'HAM - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()

# Plot of the biarticular muscle activities from joint hip/knee
for test_id in RF_r_norm.keys():
    #plotWithStd(RF_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'RF_r', 'xkcd:orange')
    plotWithStd(RF_l_norm[test_id], 'Biarticular (Hip/Knee) muscle activity (RF), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'RF - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()

# Normalized biarticular muscle activity for the joint knee/ankle (GAS)
GAS_r_norm = {}
GAS_l_norm = {}
for test_id, gas in GAS_r.items():
    GAS_r_norm[test_id] = getNormalizedData(gas, START_CYCLE_r[test_id][:-1], START_CYCLE_r[test_id][1:]-1)
for test_id, gas in GAS_l.items():
    GAS_l_norm[test_id] = getNormalizedData(gas, START_CYCLE_l[test_id][:-1], START_CYCLE_l[test_id][1:]-1)

# Plot of the biarticular muscle activity from the joint knee/ankle
for test_id in GAS_r_norm.keys():
    #plotWithStd(GAS_r_norm[test_id], '', 'Gait Cycle [%]', 'Activations [-]', 'GAS_r', 'xkcd:orange')
    plotWithStd(GAS_l_norm[test_id], 'Biarticular (Knee/Ankle) muscle activity (GAS), normalized to one stride', 'Gait Cycle [%]', 'Activations [-]', 'GAS - Exp {}'.format(test_id), test_color[test_id], 15)
    plt.axvline(56, 0, 2, color='r', linestyle='dashed')
    plt.axhline(0, color='silver', linestyle='solid')
    # INFO : Move this code outside of the for loop (Just unindent it) to display all plots in one chart
plt.legend(bbox_to_anchor = (1.05, 1), loc=2, prop={'size':14})
plt.yticks(np.arange(0, 1.2, step=0.2))  # Set y-axis to go up to 1.0 so that all the plots of the muscle activities are comparable
plt.show()